# 基于Mindspore实现二分类损失函数

通过实验了解分类损失函数原理，并能够基于Mindspore框架实现分类损失函数的计算以及训练。

## 1、实验目的
* 学会内置损失函数的使用。
* 掌握Mindspore中多种损失函数的特点和应用场景。

## 2、二分类损失函数原理介绍
损失函数（loss function）或也称为代价函数（cost function），亦称目标函数，是将随机事件或其有关随机变量的取值映射为非负实数以表示该随机事件的“风险”或“损失”的函数，用于衡量预测值与真实值差异的程度。在实际的机器学习应用中，损失函数通常会作为学习准则与优化问题相联系，通过最小化损失函数求解和评估模型。

(1) $n n$. L1Loss:计算预测值和目标值之间的平均绝对误差:
$$
\ell(x, y)=L=\left\{l_1, \ldots, l_N\right\}^{\top}, \quad \text { with } l_n=\left|x_n-y_n\right|
$$
其中N为数据集中的 batch_size 值。
$$
\ell(x, y)= \begin{cases}\operatorname{mean}(L), & \text { if reduction }=\text { 'mean'; } \\ \operatorname{sum}(L), & \text { if reduction='sum' }\end{cases}
$$
nn.L1Loss 中的参数 reduction 取值可为 mean， sum，或 none 。若 reduction 为 mean 或 sum，则输出一个经过均值或求和后的标量 Tensor (降维) ；若 reduction 为 none，则所输出Tensor的shape为广播后的shape。

(2)平均绝对误差MAE(Mean Absolute Error):计算模型预测值 f(x) 与样本真实值 y 之间距离的平均值。
$$\mathrm{MAE}=\frac{1}{m} \sum_{i=1}^{m}\left|y_{i}-f\left(x_{i}\right)\right|$$

## 3、实验环境
在动手进行实践之前，需要注意以下几点：
* 确保实验环境正确安装，包括安装MindSpore。安装过程：首先登录[MindSpore官网安装页面](https://www.mindspore.cn/install)，根据安装指南下载安装包及查询相关文档。同时，官网环境安装也可以按下表说明找到对应环境搭建文档链接，根据环境搭建手册配置对应的实验环境。
* 推荐使用交互式的计算环境Jupyter Notebook，其交互性强，易于可视化，适合频繁修改的数据分析实验环境。
* 实验也可以在华为云一站式的AI开发平台ModelArts上完成。
* 推荐实验环境：MindSpore版本=MindSpore 2.0；Python环境=3.7


|  硬件平台 |  操作系统  | 软件环境 | 开发环境 | 环境搭建链接 |
| :-----:| :----: | :----: |:----:   |:----:   |
| CPU | Windows-x64 | MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.1节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| GPU CUDA 10.1|Linux-x86_64| MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.2节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| Ascend 910  | Linux-x86_64| MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第四章](./MindSpore环境搭建实验手册.docx)|

## 4、数据处理

### 4.1数据准备

   在本次实验中，我们使用numpy从均匀分布中随机生成测试数据x，从正态分布中随机生成噪声加入因变量y。

### 4.2数据加载
使用GeneratorDataset，通过迭代列表构造数据集，指定生成数据集的列名为data和lable,使用batch函数指定每个批处理数据包含的数据条目。

In [15]:
import numpy as np
from mindspore import dataset as ds
def get_data(num, w=2.0, b=3.0):
    #生成数据及对应标签
    for _ in range(num):#num=160,生成160个样本点
        x = np.random.uniform(-10.0, 10.0)# 生成服从(-10.0, 10.0)范围内的均匀分布的元素，返回值的元素类型为浮点型。
        noise = np.random.normal(0, 1)#随机产生一个服从正态分布(0,1)的数值
        y = x * w + b + noise#增加噪音生成y
        yield np.array([x]).astype(np.float32), np.array([y]).astype(np.float32)#将数组元素类型转换为float32位;
        #我们为了提高效率，并不一次性返回所有数据，而是采用迭代器形式返回单一数据。

def create_dataset(num_data, batch_size=16):
    #加载数据集eval_data=list(get_data(5))
    train_data=list(get_data(num_data))
    dataset = ds.GeneratorDataset(train_data, column_names=['data', 'label'])
    dataset = dataset.batch(batch_size) #设置数据批次
    return dataset        
#可视化生成的数据
eval_data=list(get_data(5))
x,y=zip(*eval_data)#zip()函数迭代eval_data，将eval_data中的元素打包成一个个元组，然后返回由这些元组组成的列表。
eval_data

[(array([3.5930421], dtype=float32), array([10.325392], dtype=float32)),
 (array([7.132953], dtype=float32), array([17.879877], dtype=float32)),
 (array([5.614383], dtype=float32), array([15.581945], dtype=float32)),
 (array([-8.709806], dtype=float32), array([-13.120221], dtype=float32)),
 (array([-1.4440981], dtype=float32), array([0.6155094], dtype=float32))]

## 5、模型构建

### 5.1内置损失函数
下面介绍 mindspore. $\mathrm{nn}$ 模块中内置的损失函数L1损失。

In [16]:
# 1.内置损失函数
import mindspore.nn as nn
import mindspore as ms

# 输出loss均值
loss = nn.L1Loss()
# 输出loss和
loss_sum = nn.L1Loss(reduction='sum')
# 输出loss原值
loss_none = nn.L1Loss(reduction='none')

input_data = ms.Tensor(np.array([1, 0, 1, 0, 1, 0]).astype(np.float32))
target_data = ms.Tensor(np.array([0, 0, 1, 1, 1, 0]).astype(np.float32))

print("loss:", loss(input_data, target_data))
print("loss_sum:", loss_sum(input_data, target_data))
print("loss_none:\n", loss_none(input_data, target_data))

loss: 0.33333334
loss_sum: 2.0
loss_none:
 [1. 0. 0. 1. 0. 0.]


### 5.2基于nn.Cell构造损失函数

nn.Cell 是MindSpore的基类，不但可用于构建网络，还可用于定义损失函数。使用 $n n$.Cell定义损失函数的过程与定义一个普通的网络相似，差别在于，其执行逻辑部分要计算的是前向网络输出与真实值之间的误差。

In [17]:
# 2.基于nn.Cell构造损失函数
import mindspore.ops as ops

class MAELoss(nn.Cell):
    #自定义损失函数MAELoss

    def __init__(self):
        #初始化
        super(MAELoss, self).__init__()
        self.abs = ops.Abs()
        self.reduce_mean = ops.ReduceMean()

    def construct(self, base, target):
        #调用算子
        x = self.abs(base - target)
        return self.reduce_mean(x)

loss = MAELoss()

input_data = ms.Tensor(np.array([1, 0, 1, 0, 1, 0]).astype(np.float32))  # 生成预测值
target_data = ms.Tensor(np.array([0, 0, 1, 1, 1, 0]).astype(np.float32)) # 生成真实值

output = loss(input_data, target_data)
print(output)

0.33333334


### 5.3基于nn.LossBase构造损失函数

基于nn.LossBase构造损失函数MAELoss与基于nn.Cell构造损失函数的过程类似，都要重写__init__方法和construct方法。

nn.LossBase可使用方法get_loss将reduction应用于损失计算。

In [18]:
# 基于nn.LossBase构造损失函数
class MAELoss(nn.LossBase):
    #自定义损失函数MAELoss

    def __init__(self, reduction="mean"):
        #初始化并求loss均值
        super(MAELoss, self).__init__(reduction)
        self.abs = ops.Abs()  # 求绝对值算子

    def construct(self, base, target):
        x = self.abs(base - target)
        return self.get_loss(x)  # 返回loss均值

loss = MAELoss()

input_data = ms.Tensor(np.array([1, 0, 1, 0, 1, 0]).astype(np.float32))  # 生成预测值
target_data = ms.Tensor(np.array([0, 0, 1, 1, 1, 0]).astype(np.float32))  # 生成真实值

output = loss(input_data, target_data)
print(output)

0.33333334


## 6、模型构建

损失函数MAELoss自定义完成后，可使用MindSpore的接口Model中train接口进行模型训练，构造Model时需传入前向网络、损失函数和优化器，Model会在内部将它们关联起来，生成一个可用于训练的网络模型。

在Model中，前向网络和损失函数通过nn.WithLossCell关联起来，nn.WithLossCell支持两个输入，分别为data和label。

使用最简单的线性函数模型：f(x)=wx+b；在定义任何网络块时，需要继承nn.Cell类，来实现一些模块的基本功能，在__init__中定义网络层的基本信息，在construct中定义传播过程。

In [20]:
import mindspore as ms
from mindspore import dataset as ds
from mindspore.common.initializer import Normal
from mindspore.train import LossMonitor
class LinearNet(nn.Cell):
    #定义线性回归网络
    def __init__(self):
        super(LinearNet, self).__init__()
        # 定义一个线性层，同时初始化权重和偏置
        self.fc = nn.Dense(1, 1, Normal(0.02), Normal(0.02))#初始化输入输出维度为1，w和b
    def construct(self, x):
        return self.fc(x)

## 7、模型训练
生成num_data个数据点，实例化线性网络，选择MAELoss和Momentum优化器进行训练。

In [21]:
ds_train = create_dataset(num_data=160)
ds_train.get_dataset_size()
#print('数据集批次：',ds_train.get_dataset_size())

[WARNING] ME(21220:2916,MainProcess):2023-04-20-17:09:57.945.377 [mindspore\dataset\engine\datasets_user_defined.py:657] Python multiprocessing is not supported on Windows platform.


10

In [22]:
#将之前定义的网络实例化
net = LinearNet()

In [23]:
#因为是回归，所以我们选择MSE均方误差来计算损失
loss = MAELoss()

反向传播网络的目标就是要不断的进行更新其权重，来达到拟合我们的样本数据，从而使得我们的loss达到最小值，所以使用梯度下降的方式进行更新其权重。

In [24]:
#这里使用Momentum优化器,传入训练参数以及学习率
opt = nn.Momentum(net.trainable_params(), learning_rate=0.005, momentum=0.9)
# 使用model接口将网络、损失函数和优化器关联起来
model = ms.Model(net, loss, opt)
model.train(epoch=1, train_dataset=ds_train, callbacks=[LossMonitor(1)])

epoch: 1 step: 1, loss is 9.935867309570312
epoch: 1 step: 2, loss is 7.916937828063965
epoch: 1 step: 3, loss is 9.898558616638184
epoch: 1 step: 4, loss is 6.957692623138428
epoch: 1 step: 5, loss is 7.872587203979492
epoch: 1 step: 6, loss is 6.1877264976501465
epoch: 1 step: 7, loss is 7.800919055938721
epoch: 1 step: 8, loss is 10.210296630859375
epoch: 1 step: 9, loss is 6.097762107849121
epoch: 1 step: 10, loss is 7.093074321746826
